In [17]:
#Importing the necessary libraries
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib qt
data = pd.read_csv(
    filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
    header=None,
    sep=',')

data.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
data.dropna(how="all", inplace=True)

data.head()

,sepal_len,sepal_wid,petal_len,petal_wid,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [16]:
# Split data into X data & y classes
X = data.ix[:,0:4].values
y = data.ix[:,4].values
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)
import numpy as np
mean_vec = np.mean(X_std, axis=0)

#Eigen decomposition of the covariance matrix after standardizing the data:
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
print('Covariance matrix \n%s' %cov_mat)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
print('Eigen vectors \n%s' %eig_vecs)
print('Eigen values \n%s' %eig_vals)

#Eigen decomposition of the standardized data based on the correlation matrix:
cor_mat1 = np.corrcoef(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat1)
print('\nEigen vectors obtained after decomposition of the standardized data \n%s' %eig_vecs)
print('Eigen values are: \n%s' %eig_vals)

#Eigen decomposition of the raw data based on the correlation matrix:
cor_mat2 = np.corrcoef(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat2)
print('\n Eigen vectors obtained after decomposition of the raw data \n%s' %eig_vecs)
print('Eigen values are:\n%s' %eig_vals)

for ev in eig_vecs.T:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sorting the eigen values...
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('\n\nEigen values in descending order:\t')
for i in eig_pairs:
    print(i[0])
    
#Matrix W is a 4 X 4 matrix, but we need a 4 X 3 matrix to do multiplication with a 150 X 4 matrix
matrix_w = np.hstack((eig_pairs[0][1].reshape(4,1),eig_pairs[1][1].reshape(4,1),eig_pairs[2][1].reshape(4,1),eig_pairs[3][1].reshape(4,1)))
print('\n Original matrix is:\n',matrix_w)
#Deleting the last row so that it becomes a 4 X 3 matrix
matrix_w = np.delete(matrix_w, (3),axis=0)
print('\nMatrix W after deleting last row:\n', matrix_w)

#The previous step resulted in 3 X 4 matrix, but we need a 4 X 3 to facilitate multiplication
#Therefore we obtain a 4 X 3 matrix by transposing it...
print('\nTransposed Matrix:')
tmatrix_wnew=(np.transpose(matrix_w))
print(tmatrix_w)

Y = X_std.dot(tmatrix_w)
#print("\n The resulting matrix is: \n",Y)

Covariance matrix 
[[ 1.00671141 -0.11010327  0.87760486  0.82344326]
 [-0.11010327  1.00671141 -0.42333835 -0.358937  ]
 [ 0.87760486 -0.42333835  1.00671141  0.96921855]
 [ 0.82344326 -0.358937    0.96921855  1.00671141]]
Eigen vectors 
[[ 0.52237162 -0.37231836 -0.72101681  0.26199559]
 [-0.26335492 -0.92555649  0.24203288 -0.12413481]
 [ 0.58125401 -0.02109478  0.14089226 -0.80115427]
 [ 0.56561105 -0.06541577  0.6338014   0.52354627]]
Eigen values 
[2.93035378 0.92740362 0.14834223 0.02074601]

Eigen vectors obtained after decomposition of the standardized data 
[[ 0.52237162 -0.37231836 -0.72101681  0.26199559]
 [-0.26335492 -0.92555649  0.24203288 -0.12413481]
 [ 0.58125401 -0.02109478  0.14089226 -0.80115427]
 [ 0.56561105 -0.06541577  0.6338014   0.52354627]]
Eigen values are: 
[2.91081808 0.92122093 0.14735328 0.02060771]

 Eigen vectors obtained after decomposition of the raw data 
[[ 0.52237162 -0.37231836 -0.72101681  0.26199559]
 [-0.26335492 -0.92555649  0.24203288 -0.12

C:\Users\USERRR\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\Users\USERRR\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
#Plotting the IRIS dataset after performing PCA:

sns.set_style("white")
df = sns.load_dataset('iris')
my_dpi=96
plt.figure(figsize=(480/my_dpi, 480/my_dpi), dpi=my_dpi)
df['species']=pd.Categorical(df['species'])
my_color=df['species'].cat.codes
df = df.drop('species', 1)
pca = PCA(n_components=3)
pca.fit(df)
result=pd.DataFrame(pca.transform(df), columns=['PCA%i' % i for i in range(3)], index=df.index)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(result['PCA0'], result['PCA1'], result['PCA2'], c=my_color, cmap="Set2_r", s=60)
#ax.scatter(data['sepal_len'], data['sepal_wid'], data['petal_len'], c=my_color, cmap="Set2_r", s=60)
xAxisLine = ((min(result['PCA0']), max(result['PCA0'])), (0, 0), (0,0))
ax.plot(xAxisLine[0], xAxisLine[1], xAxisLine[2], 'r')
yAxisLine = ((0, 0), (min(result['PCA1']), max(result['PCA1'])), (0,0))
ax.plot(yAxisLine[0], yAxisLine[1], yAxisLine[2], 'r')
zAxisLine = ((0, 0), (0,0), (min(result['PCA2']), max(result['PCA2'])))
ax.plot(zAxisLine[0], zAxisLine[1], zAxisLine[2], 'r')
ax.set_xlabel("PCA1")
ax.set_ylabel("PCA2")
ax.set_zlabel("PCA3")
ax.set_title("PCA on the IRIS dataset")
plt.show()

In [2]:
# Applying KNN on IRIS data set
import csv
import random
import math
import operator

#This function is used to load the dataset
def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)-1):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            if  random.random()<split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])

#Function to calculate Euclidean distance
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

#Used to find the k-nearest neighbours
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1  #Number of attribute...
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist)) 
        distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    #Sort it in descending order based on 1st attribute...
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    #print('classVotes=',sortedVotes[0][0])
    return sortedVotes[0][0]    #Return the class name which has maximum value...

#Gets the prediction in %
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

def main():
    # prepare data
    trainingSet=[]
    testSet=[]
    split = 0.60
    loadDataset('iris.csv', split , trainingSet, testSet)

    print( 'Length of Train set: ', len(trainingSet))
    print( 'Length of Test set: ', len(testSet))
    
    # generate predictions
    predictions=[]
    k=int(input("Enter the value Of K:"))
    i=0
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        #print('predicted=' + str(result) + ', actual=' + str(testSet[x][-1]))
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: ' + str(accuracy) + '%')
    
main()

Length of Train set:  92
Length of Test set:  58
Enter the value Of K:3
Accuracy: 94.82758620689656%
